# 01 — Data Collection

**Purpose**  
Set up local dataset access and verify class distributions to ensure the project can load images reliably.

**Objectives**  
- Verify dataset folder structure (healthy / powdery_mildew).  
- Count images per class and detect missing/corrupted files.  
- Produce a compact summary table for later reference.

**Inputs**  
- Local dataset path: `../inputs/cherry_leaves_dataset/`  
  - Subfolders: `healthy/`, `powdery_mildew/`

**Outputs**  
- Summary table with image counts per class (for README and EDA planning).  
- Notes on data integrity (missing files, unexpected extensions).

**Notes**  
This notebook does *not* perform any model training. It only validates the availability and integrity of the raw data used later in EDA and modelling.